In [ ]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import numpy as np
import pandas as pd
import os
from ipywebrtc import VideoStream
plotly.tools.set_credentials_file(username=os.environ['PLOTLY_USERNAME'],                                              
                                  api_key=os.environ['PLOTLY_API_KEY'])
plotly.offline.init_notebook_mode()



In [ ]:
# get the path for all fitness_generation and fitness_population
files_vrep = sorted(Path('data/results/').glob('vrep_*/fitness_population'))
files_thymio = sorted(Path('data/results/').glob('thymio_*/fitness_population'))
f_gen = sorted(Path('data/results/').glob('*/fitness_generation_*'))

In [ ]:
files_vrep

In [ ]:
# list of dataframes of each simulation run
f_dt_list_vrep = [pd.read_csv(f, names=['gen','avg','std','max','median']) for f in files_vrep]
f_dt_list_thymio = [pd.read_csv(f, names=['gen','avg','std','max','median']) for f in files_thymio]

In [ ]:
f_dt_list_thymio

In [ ]:
# concat all the dataframes into one dataframe
f_dt_vrep = pd.concat(tuple(d for d in f_dt_list_vrep))
f_dt_thymio = pd.concat(tuple(d for d in f_dt_list_thymio))

In [ ]:
f_dt_thymio.head()

In [ ]:
# average dataframe of simulation runs
f_dt_vrep_avg = f_dt_vrep.groupby(f_dt_vrep.gen).mean()
f_dt_thymio_avg = f_dt_thymio.groupby(f_dt_thymio.gen).mean()

In [ ]:
f_dt_thymio_avg.head()

In [ ]:
q_dt_vrep = (f_dt_vrep[['max']].groupby(f_dt_vrep.gen)
                .quantile([.25, .5, .75])
                .rename(index=str, columns={'max': 'fitness'})
                .unstack(level=-2)
                .T
                .reset_index()
                .set_index('gen')
                .drop(columns='level_0')
                .rename(index=str, columns={'0.25': 'q1', '0.5': 'q2', '0.75': 'q3'})
             )
q_dt_vrep.index = q_dt_vrep.index.astype(int)

q_dt_thymio = (f_dt_thymio[['max']].groupby(f_dt_thymio.gen)
                    .quantile([.25, .5, .75])
                    .rename(index=str, columns={'max': 'fitness'})
                    .unstack(level=-2)
                    .T
                    .reset_index()
                    .set_index('gen')
                    .drop(columns='level_0')
                    .rename(index=str, columns={'0.25': 'q1', '0.5': 'q2', '0.75': 'q3'})
              )
q_dt_thymio.index = q_dt_thymio.index.astype(int)

In [ ]:
q_dt_vrep.index

In [ ]:
q_vrep = q_dt_vrep.join(f_dt_vrep_avg).sort_values(by='gen')

In [ ]:
q_vrep.head()

In [ ]:
fitness_thymio_runs.head()

In [ ]:
gen_data.head()

In [ ]:
plot_runs(f_dt_thymio_avg, 'AVG Fitness of 2 Thymio Simulation Runs')

In [ ]:
plot_runs(f_dt_vrep_avg, 'AVG Fitness of 4 Vrep Simulation Runs')

In [ ]:
# slicing the data frame
thymio_0_df = gen_data.loc[gen_data['sim'] == 'thymio_0']
vrep_0_df = gen_data.loc[gen_data['sim'] == 'vrep_0']

In [ ]:
sample = thymio_0_df.loc[thymio_0_df['gen'] == 0]

In [ ]:
sample.sort_values(by=['fitness']).head()

In [ ]:
plot_stats(thymio_0_df, 'thymio_0')

In [ ]:
plot_single_run_scatter(thymio_0_df, fitness_thymio[0], 'thymio_0')

In [ ]:
plot_single_run_scatter(vrep_0_df, fitness_vrep[0], 'vrep_0')

In [ ]:
def plot_single_run_scatter(scatter, dt, title):

    l = []
    y = []

    N = len(scatter.gen.unique())

    c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

    for i in range(int(N)):
        subset = scatter.loc[scatter['gen'] == i]

        trace0= go.Scatter(
            x= subset.loc[:, 'gen'],
            y= subset.loc[:, 'fitness'],
            mode= 'markers',
            marker= dict(size= 7,
                        line= dict(width=1),
                        color= c[i],
                        opacity= 0.5
                       ),
            name = 'gen {}'.format(i),
            text = subset.loc[:, 'genome']
        )
        l.append(trace0);

    trace0 = go.Scatter(
        x = dt.loc[:, 'gen'],
        y = dt.loc[:, 'max'],
        mode = 'lines',
        name = 'Max',
        line = dict(
            color = "rgb(204, 51, 51)", 
            dash = "solid", 
            shape = "spline", 
            smoothing = 0.0, 
            width= 2
        ),
    )

    trace1 = go.Scatter(
        x = dt.loc[:, 'gen'],
        y = dt.loc[:, 'median'],
        mode = 'lines',
        name = 'Median',
        line = dict(
            color = "rgb(173, 181, 97)",
            shape = "spline",
            dash = "solid",
            smoothing = 0.0, 
            width= 2
        )
    )

    trace2 = go.Scatter(
        x = dt.loc[:, 'gen'],
        y = dt.loc[:, 'avg'],
        mode = 'lines',
        name = 'Average',
        line = dict(
            color = "rgb(62, 173, 212)",
            shape = "spline",
            dash = "solid",
            smoothing = 0.0, 
            width= 2
        )
    )

    data = [trace0, trace1, trace2]

    layout= go.Layout(
        title= 'Fitness of Population Individuals - {}'.format(title),
        hovermode= 'closest',
        xaxis= dict(
            title= 'Generations',
            ticklen= 5,
            zeroline= False,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Fitness',
            ticklen= 5,
            gridwidth= 1,
        ),
        showlegend= False
    )

    fig = go.Figure(data=data+l, layout=layout)
    
    return py.iplot(fig, filename='single-run-scater-line-plot', layout=layout)

In [ ]:
def plot_runs(dt, title, offline=True):
    """Plots the Max/Average/Median"""
    trace0 = go.Scatter(
        x = dt.loc[:, 'gen'],
        y = dt.loc[:, 'max'],
        mode = 'lines',
        name = 'Max',
        line = dict(
            color = "rgb(204, 51, 51)", 
            dash = "solid", 
            shape = "spline", 
            smoothing = 0.0, 
            width= 2
        ),
    )

    trace1 = go.Scatter(
        x = dt.loc[:, 'gen'],
        y = dt.loc[:, 'median'],
        mode = 'lines',
        name = 'Median',
        line = dict(
            color = "rgb(173, 181, 97)",
            shape = "spline",
            dash = "solid",
            smoothing = 0.0, 
            width= 2
        )
    )

    trace2 = go.Scatter(
        x = dt.loc[:, 'gen'],
        y = dt.loc[:, 'avg'],
        mode = 'lines',
        name = 'Average',
        line = dict(
            color = "rgb(62, 173, 212)",
            shape = "spline",
            dash = "solid",
            smoothing = 0.0, 
            width= 2
        )
    )

    layout = go.Layout(
      showlegend = True,
      hovermode= 'closest',
      title = title,
      xaxis =  dict(
        autorange = False,
        range = [0, 20],
        showspikes = False,
        title = "Generations",
        ticklen= 5,
        gridwidth= 1,
      ),
      yaxis = dict(
        autorange = True,
        title = "Fitness",
        ticklen= 5,
        gridwidth= 1,
      ),
    )

    data = [trace0, trace1, trace2]
    fig = go.Figure(data, layout=layout)
    
    return py.iplot(fig, filename=title)

In [ ]:
def plot_stats(dt, title):
    """Plots a Scatter plot of each individual in the population"""
    l = []
    y = []

    N = len(dt.gen.unique())

    c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
    for i in range(int(N)):
        subset = dt.loc[dt['gen'] == i]

        trace0= go.Scatter(
            x= subset.loc[:, 'gen'],
            y= subset.loc[:, 'fitness'],
            mode= 'markers',
            marker= dict(size= 14,
                        line= dict(width=1),
                        color= c[i],
                        opacity= 0.3
                       ),
            name = 'gen {}'.format(i),
            text = subset.loc[:, 'genome'],
        )
        l.append(trace0);

    layout= go.Layout(
        title= 'Fitness of Population Individuals - {}'.format(title),
        hovermode= 'closest',
        xaxis= dict(
            title= 'Generations',
            ticklen= 5,
            zeroline= False,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Fitness',
            ticklen= 5,
            gridwidth= 1,
        ),
        showlegend= False
    )
    
    fig = go.Figure(data=l, layout=layout)
    
    return py.iplot(fig)

In [ ]:
video2 = VideoStream.from_url('data/results/thymio_to_vrep/thymio_1_best_1.mp4')
video2

In [ ]:
video2 = VideoStream.from_url('data/results/thymio_to_vrep/thymio_1_to_vrep_best_1.mp4')
video2

In [ ]:
video2 = VideoStream.from_url('data/results/thymio_to_vrep/thymio_1_best_genome_1.mp4')
video2

In [ ]:
t_best = './data/results/thymio_to_vrep/thymio_1_best_1_270_hw_simulation.txt'
t_v_best = './data/results/thymio_to_vrep/thymio_1_best_1ToVrep_270_simulation.txt'

names = ['id', 'net_left', 'net_right',
         'normalized_left', 'normalized_right',
         'sensor_activation', 'norm_sensor_activation',
         'wheel_center', 'straight_movements',
         'obstacle_distance', 's_max_activation', 'fitness_t', 'robot_position']

converters = {
    0: lambda x: int(x),
    1: lambda x: float(x),
    2: lambda x: float(x),
    3: lambda x: float(x),
    4: lambda x: float(x),
    5: lambda x: np.array(list(map(float, x.replace('[', '').replace(']', '').split())), dtype=float),
    6: lambda x: np.array(list(map(float, x.replace('[', '').replace(']', '').split())), dtype=float),
    7: lambda x: float(x),
    8: lambda x: float(x),
    9: lambda x: float(x),
    10: lambda x: float(x),
    11: lambda x: float(x),
    12: lambda x: np.array(list(map(float, x.replace('[', '').replace(']', '').split())), dtype=float),
    
}

t_dt = pd.read_csv(t_best, sep=',', names=names, converters=converters, header=None)
v_dt = pd.read_csv(t_v_best, sep=',', names=names, converters=converters, header=None)

In [ ]:
video2 = VideoStream.from_url('data/results/vrep_to_thymio/vrep_0_to_thymio_best_0.mp4')
video2

In [ ]:
video2 = VideoStream.from_url('data/results/vrep_to_thymio/vrep_0_to_thymio_best_0_transfered.mp4')
video2

In [ ]:
q_dt_vrep.index.values

In [ ]:
upper_bound = go.Scatter(
    name='75%',
    x=q_vrep.index.values,
    y=q_vrep.loc[:, 'q3'],
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

trace = go.Scatter(
    name='Median',
    x=q_vrep.index.values,
    y=q_vrep.loc[:, 'q2'],
    mode='lines',
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty')

lower_bound = go.Scatter(
    name='25%',
    x=q_vrep.index.values,
    y=q_vrep.loc[:, 'q1'],
    marker=dict(color="#444"),
    line=dict(width=0),
    mode='lines')

trace_max = go.Scatter(
    x = q_vrep.index.values,
    y = q_vrep.loc[:, 'max'],
    mode = 'lines',
    name = 'Max',
    line = dict(
        color = "rgb(204, 51, 51)", 
        dash = "solid", 
        shape = "spline", 
        smoothing = 0.0, 
        width= 2
    ),
)

data = [lower_bound, trace, upper_bound, trace_max]

layout= go.Layout(
    title= 'Fitness of Population Individuals',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Generations',
        ticklen= 5,
        zeroline= False,
        gridwidth= 1,
    ),
    yaxis=dict(
        title= 'Fitness',
        ticklen= 5,
        gridwidth= 1,
    ),
    showlegend= True
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='pandas-continuous-error-bars')

In [2]:
def plot_grid(grid):

    trace = go.Heatmap(z=grid, colorscale = 'Viridis')
    data=[trace]

    layout= go.Layout(
        title= 'Environment and obstacles',
        showlegend= False
    )

    return py.iplot(data, filename='grid-heatmap', layout=layout)